# TDSP Stage 3 – Modeling

## Proyek: Segmentasi Pelanggan Online Retail Berbasis RFM (Spark)

Stage ini berfokus pada **feature engineering RFM**, **transformasi data**, dan **clustering pelanggan** menggunakan Apache Spark.
Hasil dari stage ini adalah segmen pelanggan yang bermakna dan dapat diinterpretasikan secara bisnis.

## 3.1 Tujuan Modeling

Tujuan utama tahap modeling adalah:

1. Mengubah data transaksi menjadi **fitur RFM** pada level pelanggan.
2. Menyiapkan dataset yang siap untuk algoritma clustering.
3. Mengelompokkan pelanggan berdasarkan kemiripan perilaku transaksi.
4. Menghasilkan segmentasi yang **interpretable** dan **actionable** bagi bisnis.

Pendekatan yang digunakan bersifat **unsupervised learning**, karena tidak terdapat label target.

## 3.2 Persiapan Data Modeling

Dataset yang digunakan adalah hasil filter dari TDSP Stage 2.
Langkah awal:
- Pastikan kolom tanggal dalam format timestamp
- Hitung nilai transaksi (Amount)
- Tentukan *reference date* untuk perhitungan Recency

In [6]:
pass

## 3.3 Feature Engineering – Monetary

**Monetary** merepresentasikan total nilai uang yang dibelanjakan pelanggan.

Langkah:
- Hitung `Amount = Quantity × Price`
- Agregasi total Amount per pelanggan (PostCode)

In [7]:
pass

## 3.4 Feature Engineering – Recency & Frequency

- **Recency**: jarak waktu (hari) antara transaksi terakhir pelanggan dengan tanggal referensi
- **Frequency**: jumlah transaksi unik (Invoice) per pelanggan


In [8]:
pass

## 3.5 Pemeriksaan Distribusi RFM

Distribusi RFM biasanya **skewed**, khususnya Monetary.
Pemeriksaan statistik deskriptif penting sebelum scaling dan clustering.

In [9]:
pass

## 3.6 Feature Scaling

Karena perbedaan skala antar fitur RFM cukup besar,
maka diperlukan **normalisasi / scaling** sebelum clustering.

Metode yang digunakan:
- `StandardScaler` (mean = 0, std = 1)

In [10]:
pass

## 3.7 Clustering Pelanggan (K-Means)

Algoritma **K-Means** digunakan untuk mengelompokkan pelanggan berdasarkan fitur RFM.

Jumlah cluster ditentukan berdasarkan:
- Interpretabilitas bisnis
- Eksperimen awal

Pada tahap ini digunakan **k = 5**, sesuai praktik umum segmentasi RFM.

In [11]:
pass

## 3.8 Interpretasi Awal Cluster

Langkah awal interpretasi:
- Hitung statistik RFM per cluster
- Bandingkan karakteristik antar cluster

Interpretasi ini akan menjadi dasar rekomendasi bisnis pada stage selanjutnya.

In [12]:
pass

## Ringkasan Stage 3

Pada TDSP Stage 3 ini, kita telah:
- Melakukan feature engineering RFM menggunakan Spark
- Melakukan scaling fitur untuk clustering
- Mengelompokkan pelanggan menggunakan K-Means
- Menghasilkan segmentasi pelanggan awal

Tahap berikutnya adalah **TDSP Stage 4 – Deployment**,
yang berfokus pada penyajian hasil, insight bisnis, dan output yang siap dikonsumsi stakeholder.